In [0]:
%sql
select * from workspace.bookstore_eng_ass.customers

In [0]:
%sql
DESCRIBE workspace.bookstore_eng_ass.customers

In [0]:
%sql
select customer_id, profile:first_name, profile:address:country
from workspace.bookstore_eng_ass.customers

In [0]:
%sql
select profile from workspace.bookstore_eng_ass.customers
limit 1

In [0]:
%sql
create or replace temp view parsed_customer as 
select customer_id, from_json(profile, schema_of_json('{"first_name":"Jacquelyn","last_name":"Abby","gender":"Female","address":{"street":"0 Elmside Court","city":"Gornji Milanovac","country":"Serbia"}}')) as profile_struct
from workspace.bookstore_eng_ass.customers
    

In [0]:
%sql
select * from parsed_customer

In [0]:
%sql
describe parsed_customer

In [0]:
%sql
select customer_id, profile_struct.first_name, profile_struct.address.country 
from parsed_customer

In [0]:
%sql
create or replace temp view customer_final 
as 
select customer_id, profile_struct.* from 
parsed_customer;

select * from customer_final

In [0]:
%sql
select order_id, customer_id, books from workspace.bookstore_eng_ass.orders

In [0]:
%sql
select order_id, customer_id, explode(books) as book
from workspace.bookstore_eng_ass.orders

In [0]:
%sql
select customer_id,
collect_set(order_id) as orders_set,
collect_set(books.book_id) as books_set
from bookstore_eng_ass.orders
group by customer_id

In [0]:
%sql
select customer_id, 
collect_set(books.book_id) as before_flatten,
array_distinct(flatten(collect_set(books.book_id))) as after_flatten
from bookstore_eng_ass.orders
group by customer_id

In [0]:
%sql
create or replace view workspace.bookstore_eng_ass.orders_enriched as 
select a.order_id, a.order_timestamp, a.customer_id, a.quantity, a.total, a.books, a.book, b.book_id, b.title, b.author, b.category, b.price
from (
  select *, explode(books) as book
  from workspace.bookstore_eng_ass.orders) a
  join workspace.default.books b 
  on a.book.book_id = b.book_id